### Кукушкино хэширование
**Алгоритм разрешения коллизий** значений хеш-функций в таблице **с постоянным временем** выборки в худшем случае.<br/>
В алгоритме предусматривается возможность выталкивания старого ключа при вставке нового.<br/>
Основная идея кукушкиного хеширования заключается в разрешении коллизий путём использования двух хеш-функций вместо одной. Это обеспечивает два возможных положения в хеш-таблице для каждого ключа. <br/>
В одном из обычных вариантов алгоритма хеш-таблица разбивается на две меньшие таблицы меньшего размера и каждая хеш-функция даёт индекс в одну из этих двух таблиц.

Когда вставляется новый ключ и одна из двух ячеек пуста, ключ может быть помещён в эту ячейку. В случае же, когда обе ячейки заняты, необходимо переместить другие ключи в другие места (или, наоборот, на их прежние места), чтобы освободить место для нового ключа.

Используется **жадный алгоритм** — ключ помещается в одну из возможных позиций, «выталкивая» любой ключ, который был в этой позиции. Вытолкнутый ключ затем помещается в его альтернативную позицию, снова выталкивая любой ключ, который мог там оказаться. Процесс продолжается, пока не найдётся пустая позиция.

Возможен, однако, случай, когда процесс вставки заканчивается неудачей, попадая в **бесконечный цикл** или когда образуется слишком длинная цепочка (длиннее, чем заранее заданный **порог**, зависящий логарифмически от длины таблицы). В этом случае хеш-таблица перестраивается на месте с новыми хеш-функциями.

Ожидаемое время вставки постоянно, даже если принимать во внимание возможную необходимость перестройки таблицы, пока число ключей меньше половины ёмкости хеш-таблицы, то есть **коэффициент загрузки меньше 50 %**.

Чтобы обеспечить это, используется теория случайных графов

#### Кукушкин граф
Вершинами являются ячейки хеш-таблицы, а рёбра для каждого хешируемого соединяют два возможных положения (ячейки хеш-таблицы). 

Тогда жадный алгоритм вставки множества значений в кукушкину хеш-таблицу успешно завершается тогда и только тогда, когда кукушкин граф для этого множества значений является псевдолесом, графом максимум с одним циклом в каждой компоненте связности. 

Если хеш-функция выбирается случайно, кукушкин граф будет случайным графом

С высокой степенью вероятности для случайного графа, в котором отношение числа рёбер к числу вершин ограничено сверху 1/2, граф является псевдолесом и алгоритм кукушкиного хеширования располагает успешно все ключи. 

In [234]:
from BitHash import BitHash, ResetBitHash
import random
from IPython.display import clear_output 

In [6]:
class Node(object):
    def __init__(self, k, d):
        self.key  = k 
        self.data = d

    def __str__(self):
        return "(" + str(self.key) + ", " + str(self.data) + ")"

In [242]:
class HashTab(object):
    def __init__(self, size):
        self.__hashArray1 = [None] * (size // 2)  # создаем 2 массива в половину размера хеш-таблицы
        self.__hashArray2 = [None] * (size // 2) 
        self.__size = size                  # размер таблицы (обоих массивов)

        self.__loopLimit = 15                # порог поиска места

        self.__numRecords = 0               # счетчик нод
        self.collision = 0                  # счетчик коллизий
  
    def __len__(self): return self.__numRecords

    def fullness(self): return f"{(self.__numRecords/self.__size):.2f}"

    # Хеш - функция
    def __hashFunc(self, s):
        x = BitHash(s)         
        y = BitHash(s, x)

        size = self.__size // 2  
        
        return x % size, y % size

    # вставка (возвращает true или false о вставке)
    def insert(self, k, d):
        if self.find(k) != None:  return False   # если уже есть, возвращаем фалс (не дублируем)
        
        new = Node(k, d)                           # создаем новую ноду

        # Проверяем на заполненость, иначе перестраиваем 
        if self.__numRecords >= (self.__size // 2):
            self.__growHash() 

        position1, position2 = self.__hashFunc(new.key)  # считаем хеш-функции, получаем два места

        
        pos = position1 # запоминаем первое место, для проверки на зацикливание
        table = self.__hashArray1

        # начинаем искать место в первой таблице   
        for i in range(self.__size):
            
            if table[pos] == None:               # если позиция в текущей таблице пуста
                table[pos] = new                 # вставляем ноду и выходим
                self.__numRecords += 1
                return True
            
            
            new, table[pos] = table[pos], new       # иначе выпихиваем существующий элемент и работаем уже с ним
                                                
            if pos == position1:                              # если вернулись в первое место 
                position1, position2 = self.__hashFunc(new.key) # получаем места из выпихнутой ноды,
                pos = position2                               # получаем второе место 
                table = self.__hashArray2                     # ищем во второй хеш таблице  
            else:                                             # если вернулись во вторую ноду 
                position1, position2 = self.__hashFunc(new.key) # получаем места из выпихнутой ноды
                pos = position1                              # получаем первое место
                table = self.__hashArray1                     # ищем в первой хеш таблице 

        self.collision += 1
        # если вставить не получилось        
        #self.__growHash()               #?
        self.__rehash(self.__size)        # выбираем новые хеш-функции                   
        self.insert(new.key, new.data)  # снова пытаемся вставить элемент 

        return True

    
    def __str__(self):
        str1 = "Table 1: [ " + str(self.__hashArray1[0]) 
        str2 = "\n\nTable 2: [ " + str(self.__hashArray2[0]) 
        for i in range(1, self.__size//2):
            str1 += ", " + str(self.__hashArray1[i])
        str1 += "]"

        for i in range(1, self.__size//2):
           str2 += ", " + str(self.__hashArray2[i]) 
        str2 += "]"
        
        return str1 + str2 

    # сбрасываем хеш-функцию и заново все вставляем
    def __rehash(self, size):
        print(self.__numRecords)
        clear_output(wait=False)
        ResetBitHash()          # сбрасываем хеш-функцию
        
        temp = HashTab(size)    # новая хеш-таблица

        # по новой вставляем все непустые элементы в хеш-таблицу
        for i in range(self.__size // 2):
            x = self.__hashArray1[i]
            y = self.__hashArray2[i]
            if x != None:
                temp.insert(x.key, x.data)
            if y != None:
                temp.insert(y.key, y.data)

        # сохраняем новые данные
        self.__hashArray1 = temp.__hashArray1
        self.__hashArray2 = temp.__hashArray2
        self.__numRecords = temp.__numRecords
        self.__size = temp.__size

        #self.collision = temp.collision
      
        
    # Увеличиваем размер таблицы в два раза
    def __growHash(self):
        newSize = self.__size * 2
        # поновой заполняем таблицу
        self.__rehash(newSize)

   
    def find(self, k):
        pos1, pos2 = self.__hashFunc(k)               
        x = self.__hashArray1[pos1]                 
        y = self.__hashArray2[pos2]                 
        if x != None and x.key == k:  return x.data
        if y != None and y.key == k:  return y.data
    
        # None если не нашли     
        return None

    # удаляет ноду и возвращает True при успехе
    def delete(self, k):
        pos1, pos2 = self.__hashFunc(k)  
        x = self.__hashArray1[pos1]
        y = self.__hashArray2[pos2]
        if  x != None and  x.key == k:  self.__hashArray1[pos1] = None
        elif y != None and y.key == k:  self.__hashArray2[pos2] = None
        else:   return False   # такого ключа нет 
        self.__numRecords -= 1 
        return True

In [235]:
class HashTabOneDim(object):
    def __init__(self, size):
        self.__hashArray1 = [None] * (size)  # создаем 2 массива в половину размера хеш-таблицы
        self.__size = size                  # размер таблицы (обоих массивов)

        self.__loopLimit = 25                # порог поиска места

        self.__numRecords = 0               # счетчик нод
        self.collision = 0                  # счетчик коллизий
  
    def __len__(self): return self.__numRecords

    def fullness(self): return f"{(self.__numRecords/self.__size):.2f}"

    # Хеш - функция
    def __hashFunc(self, s):
        x = BitHash(s)         
        y = BitHash(s, x)

        size = self.__size  
        
        return x % size, y % size

    # вставка (возвращает true или false о вставке)
    def insert(self, k, d):
        if self.find(k) != None:  return False   # если уже есть, возвращаем фалс (не дублируем)
        
        new = Node(k, d)                           # создаем новую ноду

        # Проверяем на заполненость, иначе перестраиваем 
        if self.__numRecords >= (self.__size // 2):
            self.__growHash() 

        position1, position2 = self.__hashFunc(new.key)  # считаем хеш-функции, получаем два места

        
        pos = position1 # запоминаем первое место, для проверки на зацикливание
        table = self.__hashArray1

        # начинаем искать место в первой таблице   
        for i in range(self.__size):
            
            if table[pos] == None:               # если позиция в текущей таблице пуста
                table[pos] = new                 # вставляем ноду и выходим
                self.__numRecords += 1
                return True
            
            
            new, table[pos] = table[pos], new       # иначе выпихиваем существующий элемент и работаем уже с ним
                                                
            if pos == position1:                              # если вернулись в первое место 
                position1, position2 = self.__hashFunc(new.key) # получаем места из выпихнутой ноды,
                pos = position2                               # получаем второе место 
                #table = self.__hashArray2                     # ищем во второй хеш таблице  
            else:                                             # если вернулись во вторую ноду 
                position1, position2 = self.__hashFunc(new.key) # получаем места из выпихнутой ноды
                pos = position1                              # получаем первое место
                #table = self.__hashArray1                     # ищем в первой хеш таблице 

        self.collision += 1
        # если вставить не получилось        
        self.__growHash()               #?
        #self.__rehash(self.__size)        # выбираем новые хеш-функции                   
        self.insert(new.key, new.data)  # снова пытаемся вставить элемент 

        return True

    
    def __str__(self):
        str1 = "Table : [ " + str(self.__hashArray1[0]) 
        
        for i in range(1, self.__size//2):
            str1 += ", " + str(self.__hashArray1[i])
        str1 += "]"

        return str1 

    # сбрасываем хеш-функцию и заново все вставляем
    def __rehash(self, size):
        print(self.__numRecords)
        clear_output(wait=False)
        ResetBitHash()          # сбрасываем хеш-функцию
        
        temp = HashTabOneDim(size)    # новая хеш-таблица

        # по новой вставляем все непустые элементы в хеш-таблицу
        for i in range(self.__size):
            x = self.__hashArray1[i]
            if x != None:
                temp.insert(x.key, x.data)
            

        # сохраняем новые данные
        self.__hashArray1 = temp.__hashArray1
        self.__numRecords = temp.__numRecords
        self.__size = temp.__size

        #self.collision = temp.collision
      
        
    # Увеличиваем размер таблицы в два раза
    def __growHash(self):
        newSize = self.__size * 2
        # поновой заполняем таблицу
        self.__rehash(newSize)

   
    def find(self, k):
        pos1, pos2 = self.__hashFunc(k)               
        x = self.__hashArray1[pos1]                 
        y = self.__hashArray1[pos2]                 
        if x != None and x.key == k:  return x.data
        if y != None and y.key == k:  return y.data
    
        # None если не нашли     
        return None

    # удаляет ноду и возвращает True при успехе
    def delete(self, k):
        pos1, pos2 = self.__hashFunc(k)  
        x = self.__hashArray1[pos1]
        y = self.__hashArray1[pos2]
        if  x != None and  x.key == k:  self.__hashArray1[pos1] = None
        elif y != None and y.key == k:  self.__hashArray1[pos2] = None
        else:   return False   # такого ключа нет 
        self.__numRecords -= 1 
        return True

In [249]:
def test():
    size = 50000
    missing = 0
    found = 0 
    
    # создаем маленькую хеш-таблицу
    c = HashTabOneDim(100)
    # c = HashTab(100)
    # Теперь вставляем size пар ключ/значение, где ключ - объединение
    # "foobarbaz" и i, и значения i
    inserted = 0
    for i in range(size): 
        if c.insert(str(i)+"foobarbaz", i):
            inserted += 1
    print("Вставлено ", inserted, " нод успешно")
    print("Заполненость таблиц ", c.fullness())
    print("Количество колизий ", c.collision, 'или ', f'{c.collision/size*100:.2f}', 'на 100 элеметов')    
        
    # Убеждаемся, что все пары ключевых данных, которые мы вставили, можно найти в хэш-таблице.
    # Это гарантирует, что изменение размера количества сегментов не приведет к потере пар ключ/данные.
    for i in range(size):
        ans = c.find(str(i)+"foobarbaz")
        if ans == None or ans != i:
            print(i, "Не найден ключ ", str(i)+"foobarbaz")
            missing += 1
            
    print("Потеряно ", missing, " нод кукушкой")
    
    # Проверяет, что все пары ключевых данных успешно удаляются 
    for i in range(size): 
        c.delete(str(i)+"foobarbaz")
        
    for i in range(size): 
        ans = c.find(str(i)+"foobarbaz") 
        if ans != None or ans == i: 
            print(i, "Невозможно удалить ", str(i)+"foobarbaz") 
            found += 1
    print("Найдено ", found, "нод, которые не удалены из кукушки") 
    
   

In [187]:
def main():
    size = 90
    missing = 0
    found = 0 
    
    # создаем маленькую хеш-таблицу
    c = HashTab(10)
    
    # Теперь вставляем size пар ключ/значение, где ключ - объединение
    # "foobarbaz" и i\im, и значения i
    inserted = 0
    for i in range(size): 
        if c.insert(str(i)+"foobarbaz", i):
            inserted += 1
    print("Вставлено ", inserted, " нод успешно")
    print("Заполненость таблиц ", c.fullness())
    print("Количество колизий ", c.collision, 'или ', f'{c.collision/size*100:.2f}', 'на 100 элеметов')    
        
    print('\n' + str(c))

In [250]:
if __name__ == '__main__':
    test()  

KeyboardInterrupt: 

#### Улучшения
* Использование трёх хеш-функций повышает коэффициент загрузки до 91 % 
* Использование двух ключей на ячейку (блочное кукушкино хеширование) позволяет повысить загрузку выше 80 %